### About

Esta notebook es parte de una estrategia de sumar explicatividad al modelo. Tomando el output de la matriz de forcast de prophet, obtenemos el peso absoluto que asigna a cada regresor. Haciendo algunas transformaciones podemos interpretar que regresores esta usando el modelo, día a día, para realizar la predicción. Tambien nos permite filtrar regresores que estan impactando poco en el modelo, simplificándolo.

In [1]:
#basics
import pandas as pd
import warnings
import itertools
import numpy as np
from datetime import datetime
import calendar

#graphs
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

#funciones externas
import sys
sys.path.append('../..')
from src import funciones
from src import prophetaux

#set options
pd.set_option('display.float_format', '{:.2f}'.format)
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 200)

#metrics
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import os
import glob
import pandas as pd


In [ ]:
os.chdir("predicciones/")
extension = 'csv'

In [3]:
#concatenamos el output-predict de todos los modelos
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
results = pd.concat([pd.read_csv(f) for f in all_filenames ])
sel_cols = [c for c in results.columns if c[-5:] != 'lower' and c[-5:] != 'upper']
results = results[sel_cols]

In [4]:
models = results.model.unique()
model_dict = {}
for x in models:
    model_dict["{0}".format(x)] = results[results.model == x]
model_dict.keys()

dict_keys(['MB_2', 'AB_3', 'MB_3', 'AB_2', 'MB_1b', 'AB_1b'])

In [ ]:
#visualizamos los regresores que aportan (al menos una vez), mas del 2% en peso relativo para el predict. Los que nunca aportan más, también son candidatos a salir del modelo.

importance = abs(model_dict['MB_2'].drop(columns=cols_no_graph+['ds']).divide(model_dict[i].yhat, 
                                                                     axis=0)).describe().transpose().sort_values(by='max',ascending=False)

features_to_plot = importance[importance['max'] > 0.02].index.drop('yhat')

#model_dict['AB_1_dic2020'][features_to_plot]
features_to_plot = features_to_plot.append(pd.Index(['ds']))
model_dict[i][features_to_plot].drop(columns='ds').divide(model_dict[i][features_to_plot].drop(columns='ds').sum(axis=1), axis='index')

In [10]:
#columnas que no queremos incluir en el plot
cols_no_graph = ['trend', 'additive_terms','extra_regressors_additive','weekly','multiplicative_terms','yearly','Unnamed: 0','sem_dia', 'model']

In [13]:
#iteramos sobre le diccionario y graficamos los regresores con plotly express.

for i in model_dict:
    #obtenemos la importancia de las features por el valor maximo aportado respecto a yhat
    importance = abs(model_dict[i].drop(columns=cols_no_graph+['ds']).divide(model_dict[i].yhat, 
                                                                    axis=0)).describe().transpose().sort_values(by='max',ascending=False)
    #graficamos solo las variables uqe hayan aportado mas de 2%
    features_to_plot = importance[importance['max'] > 0.02].index.drop('yhat').append(pd.Index(['ds']))
    
    #aplicamos el filtro y melteamos
    graph_df = model_dict[i][features_to_plot]
    graph_df = pd.melt(graph_df, id_vars =['ds']) 
    fig = px.bar(graph_df, x="ds", y="value", title= str(i) +" - aporte por regresor (>2%)", color='variable', barmode='relative')
    
    fig.write_html(str(i) + "stacked_barplot.html")
    
    importance.to_csv(str(i) + 'importance.csv')
    
    fig.show()

In [132]:
for i in model_dict:
    #obtenemos la importancia de las features por el valor maximo aportado respecto a yhat
    importance = abs(model_dict[i].drop(columns=cols_no_graph+['ds']).divide(model_dict[i].yhat, 
                                                                     axis=0)).describe().transpose().sort_values(by='max',ascending=False)
    #graficamos solo las variables que hayan aportado mas de 2%
    features_to_plot = importance[importance['max'] > 0.02].index.drop('yhat').append(pd.Index(['ds']))
    
    #aplicamos el filtro y melteamos
    graph_df = model_dict[i][features_to_plot].drop(columns='ds').divide(model_dict[i][features_to_plot].drop(columns='ds').sum(axis=1), axis='index')
    graph_df['ds'] = model_dict[i].ds
    graph_df = pd.melt(graph_df, id_vars =['ds']) 
    fig = px.bar(graph_df, x="ds", y="value", title= str(i) +" - aporte por regresor (>2%)", color='variable', barmode='relative')
    
    fig.write_html(str(i) + "stacked_barplot.html")

    fig.show()